In [148]:
import numpy as np
import pandas as pd

color_est = pd.read_csv("../data/color2emotion_rescaling.csv",index_col = False)
emotion_est = pd.read_csv("../result/emotion_estimation_hand_craft.csv")
action_est = pd.read_csv("../result/action_estimation.csv")

In [149]:
#color_est

In [150]:
emotion_est


,감정,에너지,회복,순환,정화
0,행복,0.300379,0.469413,0.287386,0.282726
1,열정,1.000000,0.598599,0.395890,0.400756
2,두려움,0.140877,0.477591,0.057623,0.300837
3,분노,0.364110,0.405571,0.323333,0.367194
4,우울,0.341376,1.000000,0.250951,0.360276
5,슬픔,0.191463,0.461462,0.185052,0.320086


In [151]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.400750,0.455728,0.589044,0.325155
1,운동,0.364873,0.483403,0.383764,0.324081
2,산책,0.320345,0.461215,0.271852,0.328704
3,수면,0.281741,0.407013,0.238239,0.359288
4,독서,0.326346,0.485431,0.260725,0.255343


In [152]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr


def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [181]:
# 감정 -> 효능 <- 해동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
             
        
    def sim_item_MAE(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(em_est - act_est))
            
            sim_dict[act_name] = sim
            
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Cosine(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = dot(em_est, act_est)/(norm(em_est)*norm(act_est))
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 Cosine 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Pearson(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = pearson_similarity(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 피어슨 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=True ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_callback(self, em_idx,callback,reverse):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = callback(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 callback 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=reverse ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    


    def max_item_MAE(self, em_idx, n):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        effs = ['에너지','회복','순환','정화']
        em_dict = {}
        i = 0
        for eff in effs:
            em_dict[eff] = em_est[i]
            i = i + 1
        max_dict = sorted(em_dict.items(), reverse=True,key= lambda x:x[1])[:n]

        sort_em_list = []
        sort_em_est = []
        for x, y in max_dict:
            sort_em_list.append(x)
            sort_em_est.append(y)

        sort_em_df = self.emothon_df[sort_em_list] 
        em_est = np.array(sort_em_df.iloc[em_idx])
        sort_act_df = self.action_df[sort_em_list]
        print(f'{em_name}의 MAX 차원 {sort_em_list}')


        sim_dict = {}
        for idx in sort_act_df.index:
            act_name , *_ = self.action_df.iloc[idx]
            act_est = np.array(sort_act_df.iloc[idx])
            sim = np.mean(np.abs(em_est - act_est))
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

            

    def complete_item(self, em_idx, n):
        pass

In [191]:
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,4)
    print("\n\n")

행복의 MAX 차원 ['회복', '에너지', '순환', '정화']
행복과 명상의 MAE : 0.11453574895858767
행복과 운동의 MAE : 0.05405441919962567
행복과 산책의 MAE : 0.022419507304827377
행복과 수면의 MAE : 0.05168657253185907
행복과 독서의 MAE : 0.024007445822159454
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['에너지', '회복', '정화', '순환']
열정과 명상의 MAE : 0.25271882116794586
열정과 운동의 MAE : 0.20978081723054248
열정과 산책의 MAE : 0.2532823234796524
열정과 수면의 MAE : 0.27724098414182663
열정과 독서의 MAE : 0.2668498965601126
--------------------
열정에서 가장 유사한 item은 운동



두려움의 MAX 차원 ['회복', '정화', '에너지', '순환']
두려움과 명상의 MAE : 0.2093686740845442
두려움과 운동의 MAE : 0.14479852529863518
두려움과 산책의 MAE : 0.10948498360812664
두려움과 수면의 MAE : 0.11262693069875239
두려움과 독서의 MAE : 0.11047627218067646
--------------------
두려움에서 가장 유사한 item은 산책



분노의 MAX 차원 ['회복', '정화', '에너지', '순환']
분노과 명상의 MAE : 0.0986365601420403
분노과 운동의 MAE : 0.04553483178218208
분노과 산책의 MAE : 0.04734467715024948
분노과 수면의 MAE : 0.04420273005962373
분노과 독서의 MAE : 0.0730207102994124
--------------------
분노에서 가장 유사한 

In [189]:
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_MAE(i)
    print('\n')


행복과 명상의 MAE : 0.11453574895858767
행복과 운동의 MAE : 0.05405441919962567
행복과 산책의 MAE : 0.022419507304827377
행복과 수면의 MAE : 0.05168657253185907
행복과 독서의 MAE : 0.024007445822159454
--------------------
행복에서 가장 유사한 item은 산책


열정과 명상의 MAE : 0.25271882116794586
열정과 운동의 MAE : 0.20978081723054248
열정과 산책의 MAE : 0.2532823234796524
열정과 수면의 MAE : 0.2772409841418267
열정과 독서의 MAE : 0.2668498965601126
--------------------
열정에서 가장 유사한 item은 운동


두려움과 명상의 MAE : 0.20936867408454418
두려움과 운동의 MAE : 0.1447985252986352
두려움과 산책의 MAE : 0.10948498360812665
두려움과 수면의 MAE : 0.11262693069875239
두려움과 독서의 MAE : 0.11047627218067646
--------------------
두려움에서 가장 유사한 item은 산책


분노과 명상의 MAE : 0.0986365601420403
분노과 운동의 MAE : 0.04553483178218208
분노과 산책의 MAE : 0.04734467715024948
분노과 수면의 MAE : 0.04420273005962373
분노과 독서의 MAE : 0.0730207102994124
--------------------
분노에서 가장 유사한 item은 수면


우울과 명상의 MAE : 0.24421456456184387
우울과 운동의 MAE : 0.1772751708825429
우울과 산책의 MAE : 0.1530719747145971
우울과 수면의 MAE : 0.16658060997724533
우울과 독서의 

In [156]:
XYZ = emotion2item(action_est, emotion_est)
print("Pearson Similarity")
for i in range(0,5):
    XYZ.sim_item(i, pearson_similarity)
    print('\n')


Pearson Similarity


AttributeError: 'emotion2item' object has no attribute 'sim_item'